In [ ]:
from bs4 import BeautifulSoup
# import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent

import time
import random
import csv

In [ ]:
links = []
property_codes =[]

## ImmoVlan

In [ ]:
# get_list_links definition function:
def get_listing_links(driver, url:str) -> None:

    #options.add_argument("disable-infobars")

    #service = ChromeService(executable_path=ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    time.sleep(random.uniform(3,7))

    # Click the cookie banner if present
    try:
        # shadow_host = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root")
        # shadow_root1 = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
        # accept_button = shadow_root1.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")
        accept_button = driver.find_element(By.ID, "didomi-notice-agree-button")
        accept_button.click()

    except Exception as e:
        print(f"{e}: Cookie banner not found")

    
    try:
        not_now_button = driver.find_element(By.CLASS_NAME, "button-link")
        not_now_button.click()
    except Exception as e:
        print(f"{e} application banner not found")
                            
   
    page= driver.page_source  

    # Transforming to BeautifulSoup
    the_soup = BeautifulSoup(page, 'html.parser') 

    # Getting all the links from the page
    
    for card in the_soup.find_all(class_="card-title ellipsis pr-2 mt-1 mb-0"):
        print(card)
        
        link = card.find('a').get('href')
        print(link)
        links.append(link)
    

In [ ]:
# Agent for Random Assignemnt
ua = UserAgent()

# Options for Driver
options = Options() 
options.add_argument("start-maximized")
options.add_argument(f"user-agent={ua.random}")
options.add_argument("--enable-javascript") 
options.add_experimental_option("excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Selenium Driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})
provinces =['antwerp', 'vlaams-brabant', 'brabant-wallon','east-flanders','west-flanders','hainaut','brussels','liege','limburg','namur','luxembourg' ]

province = 'luxembourg'
for page in range(1,51):
    #listing_url = f'https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&islifeannuity=no&page={str(page)}&noindex=1'
    listing_url = f"https://immovlan.be/en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house,apartment&provinces={province}&islifeannuity=no&page={str(page)}&noindex=1"
    
    get_listing_links(driver, listing_url)

In [ ]:
links

In [ ]:
driver.close()

In [ ]:
with open("listing_links.csv", "w") as file:
    csv_writer = csv.writer(file)
    for link in links:
        csv_writer.writerow([link])

#### Merging of all csv files to complete csv

In [1]:

import os
import pandas as pd

current_directory = os.getcwd()
print(current_directory)
csv_file_list = [file for file in os.listdir(current_directory) if file.endswith('.csv')]
print(csv_file_list)

dfs= []

for file in csv_file_list:
    df = pd.read_csv(file, names=['link'])
    dfs.append(df)

complete_df = pd.concat(dfs, ignore_index=True)
complete_df =complete_df[1:]
complete_df 


c:\Users\herms\Desktop\BeCode-Bauman\TheHill_Projects\immo-eliza-data-scraper
['complete_listing_links.csv', 'listing_links-1-3-prov.csv', 'listing_links-10-11.csv', 'listing_links-4-6-prov.csv', 'listing_links-7-9-prov.csv']


,link
1,https://immovlan.be/en/projectdetail/1485771-0...
2,https://immovlan.be/en/detail/residence/for-sa...
3,https://immovlan.be/en/detail/residence/for-sa...
4,https://immovlan.be/en/detail/apartment/for-sa...
5,https://immovlan.be/en/detail/apartment/for-sa...
...,...
21996,https://immovlan.be/en/detail/residence/for-sa...
21997,https://immovlan.be/en/detail/residence/for-sa...
21998,https://immovlan.be/en/detail/residence/for-sa...
21999,https://immovlan.be/en/detail/residence/for-sa...


##### Removing all links concerning to projects

In [2]:
final_df = complete_df[~complete_df['link'].str.contains('project')]
#final_df =final_df[1:]
final_df

,link
2,https://immovlan.be/en/detail/residence/for-sa...
3,https://immovlan.be/en/detail/residence/for-sa...
4,https://immovlan.be/en/detail/apartment/for-sa...
5,https://immovlan.be/en/detail/apartment/for-sa...
6,https://immovlan.be/en/detail/apartment/for-sa...
...,...
21996,https://immovlan.be/en/detail/residence/for-sa...
21997,https://immovlan.be/en/detail/residence/for-sa...
21998,https://immovlan.be/en/detail/residence/for-sa...
21999,https://immovlan.be/en/detail/residence/for-sa...


In [5]:

final_df.to_csv("final_listing_links.csv", index=False)



## Immoweb

In [ ]:
# get_list_links definition function:
def get_listing_links(driver, url:str) -> None:

    #options.add_argument("disable-infobars")

    #service = ChromeService(executable_path=ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    time.sleep(random.uniform(3,8))

    # Click the cookie banner if present
    try:
        shadow_host = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root")
        shadow_root1 = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
        accept_button = shadow_root1.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")
        accept_button.click()
    except Exception as e:
        print(f"{e}: Cookie banner not found")
                            
   
    page= driver.page_source  

    # Transforming to BeautifulSoup
    the_soup = BeautifulSoup(page, 'html.parser') 

    # Getting all the links from the page
    
    for card in the_soup.find_all(class_="card__title-link"):
        link = card.get('href')
        links.append(link)
        property_codes.append(link.rsplit('/', 1)[-1])


In [ ]:
# Agent for Random Assignemnt
ua = UserAgent()

# Options for Driver
options = Options() 
options.add_argument("start-maximized")
options.add_argument(f"user-agent={ua.random}")
options.add_argument("--enable-javascript") 
options.add_experimental_option("excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Selenium Driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
})

for page in range(1,4):
    first_part_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&isALifeAnnuitySale=false&page='
    listing_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page='+str(page)+'&orderBy=relevance'

    get_listing_links(driver, listing_url)

In [ ]:
# Creating the URLs
for page in range(1,4):
    first_part_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&isALifeAnnuitySale=false&page='
    listing_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page='+str(page)+'&orderBy=relevance'

    get_listing_links(listing_url)

### Code for one page

In [ ]:
## house_and_appt = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale'

In [ ]:
first_part_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&isALifeAnnuitySale=false&page='

In [ ]:
number_pages = [1,2,3]
page_number = 1

In [ ]:
listing_url = 'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page='+str(page_number)+'&orderBy=relevance'
listing_url

In [ ]:

options = Options()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--enable-javascript") 
options.add_experimental_option("excludeSwitches", ['enable-automation'])
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
url = listing_url

driver.get(url)
time.sleep(5)

shadow_host = driver
#ok_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[data-testid = 'uc-accept-all-button']")))

shadow_host = driver.find_element(By.CSS_SELECTOR, "#usercentrics-root")
shadow_root1 = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
accept_button = shadow_root1.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")
accept_button.click()
                           
print("SSSSSSSSSSSSSSSSS")
page= driver.page_source 
page


In [ ]:
the_soup = BeautifulSoup(page, 'html.parser') 
the_soup

In [ ]:
links = []
for card in the_soup.find_all(class_="card__title-link"):
    links.append(card.get('href'))

In [ ]:
for link in links:
    print(link)

In [ ]:
property_codes =[]
for link in links:
    property_codes.append(link.rsplit('/', 1)[-1])

### Comprising in dict

In [ ]:
links_with_code = dict(zip(property_codes, links))

In [ ]:
for k, v in links_with_code.items():
    print(f"code: {k} for property {v}")

In [ ]:
driver.close()